In [19]:
with open("processed_data/motions_part0.npy", "rb") as f:
    header = f.read(6)
    print(header)



b'\x93NUMPY'


In [13]:
import numpy as np

motions = np.load("processed_data/motions_part0.npy", mmap_mode='r')  # 正确方式
texts = np.load("processed_data/texts_part0.npy", mmap_mode='r')

print("motions shape:", motions.shape)
print("texts shape:", texts.shape)


motions shape: (10000, 196, 263)
texts shape: (10000, 64)


In [2]:
import xml.etree.ElementTree as ET

# 设定87个关节的名称（可根据 SMPL-X 标准替换）
joint_names = [f"j{i}" for i in range(87)]

def create_joint(name, axis):
    return ET.Element("joint", {
        "name": name,
        "type": "slide",
        "axis": axis
    })

def create_geom(name, size="0.02", rgba="0.8 0.3 0.3 1"):
    return ET.Element("geom", {
        "name": f"{name}_geom",
        "type": "sphere",
        "size": size,
        "rgba": rgba
    })

def create_body(name):
    body = ET.Element("body", {"name": name, "pos": "0 0 0"})
    body.append(create_joint(f"{name}_x", "1 0 0"))
    body.append(create_joint(f"{name}_y", "0 1 0"))
    body.append(create_joint(f"{name}_z", "0 0 1"))
    body.append(create_geom(name))
    return body

def build_model():
    mjcf = ET.Element("mujoco", {"model": "humanoid_263"})
    ET.SubElement(mjcf, "compiler", {"angle": "radian", "coordinate": "local"})
    
    default = ET.SubElement(mjcf, "default")
    ET.SubElement(default, "joint", {"limited": "false", "damping": "1"})
    ET.SubElement(default, "geom", {"type": "sphere", "size": "0.02"})

    asset = ET.SubElement(mjcf, "asset")
    ET.SubElement(asset, "texture", {
        "name": "tex", "type": "2d", "builtin": "checker", "width": "100", "height": "100"
    })
    ET.SubElement(asset, "material", {
        "name": "mat", "texture": "tex", "reflectance": "0.2"
    })

    worldbody = ET.SubElement(mjcf, "worldbody")

    root = ET.SubElement(worldbody, "body", {"name": "root", "pos": "0 0 1"})
    root.append(ET.Element("joint", {"name": "root_tx", "type": "slide", "axis": "1 0 0"}))
    root.append(ET.Element("joint", {"name": "root_ty", "type": "slide", "axis": "0 1 0"}))
    root.append(ET.Element("joint", {"name": "root_tz", "type": "slide", "axis": "0 0 1"}))
    root.append(ET.Element("geom", {"name": "root_geom", "type": "sphere", "size": "0.05", "rgba": "0.3 0.3 0.9 1"}))

    parent = root
    for name in joint_names:
        body = create_body(name)
        parent.append(body)
        parent = body  # 线性连接

    return ET.ElementTree(mjcf)

tree = build_model()
tree.write("assets/humanoid_263.xml", encoding="utf-8", xml_declaration=True)
